Beharrezko import eta bestelakoak:

In [1]:
!pip install adjustText

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for adjustText: filename=adjustText-0.7.3-py3-none-any.whl size=7096 sha256=f0704f14f32584cb4c04af6214adccaf235a8dfe830a812da0c3316448f731f8
  Stored in directory: /root/.cache/pip/wheels/06/b2/e0/b239bd2c0ff89b98c4f0c86a63e7059f38a8dfaf6df2641c63
Successfully built adjustText


In [2]:
import os
import sys
import copy
from pathlib import Path

import scipy.io
import numpy as np
from tqdm import tqdm
from adjustText import adjust_text
from matplotlib import pyplot as plt
from matplotlib.patches import Circle
import time as tm
import shutil
import matplotlib.pyplot as plt
from imageio import imread
from PIL import Image
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, Subset, DataLoader

In [3]:
!wget https://datasets.d2.mpi-inf.mpg.de/andriluka14cvpr/mpii_human_pose_v1.tar.gz  -o human_pose.zip

In [15]:
!mkdir '/content/pose_data/'
!tar -xvf  '/content/mpii_human_pose_v1.tar.gz' -C '/content/pose_data/'

Se han truncado las últimas 5000 líneas del flujo de salida.
./images/019279044.jpg
./images/068025069.jpg
./images/043439489.jpg
./images/065199085.jpg
./images/043180510.jpg
./images/077936327.jpg
./images/032166247.jpg
./images/085835606.jpg
./images/028663685.jpg
./images/085061606.jpg
./images/045571938.jpg
./images/052659494.jpg
./images/040132544.jpg
./images/041967229.jpg
./images/070550266.jpg
./images/006644163.jpg
./images/004278327.jpg
./images/061707653.jpg
./images/046332890.jpg
./images/038161650.jpg
./images/014551548.jpg
./images/012610270.jpg
./images/030097552.jpg
./images/090687283.jpg
./images/092028594.jpg
./images/020276921.jpg
./images/086201772.jpg
./images/081240248.jpg
./images/045623651.jpg
./images/002182340.jpg
./images/009557537.jpg
./images/039847876.jpg
./images/001289747.jpg
./images/062442493.jpg
./images/020601740.jpg
./images/023538326.jpg
./images/087342324.jpg
./images/061328639.jpg
./images/045247005.jpg
./images/043498418.jpg
./images/087985677.

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
root = Path(os.getcwd()).parent
sys.path.append(root)

dataset_name = ['mpii']
dataset_path = list(map(lambda x: os.path.join(root, '/content/drive/MyDrive/AA3/IANS/IANS_Pose/Proba/MPII-Human-Pose-Visualization-master/data', x), dataset_name))

# Format of filenames = [[mpii_img_1, mpii_img_2, ... (mpii_img_k)]]
#filenames_ = list(map(lambda path, name: open(os.path.join(path, '{}_filenames.txt'.format(name))), dataset_path, dataset_name))
#filenames = list(map(lambda f: f.read().split(), filenames_))
#_ = list(map(lambda f: f.close(), filenames_))

In [9]:
matlab_mpii = scipy.io.loadmat(os.path.join(dataset_path[0], 'joints.mat'), struct_as_record=False)['RELEASE'][0, 0]
num_images = annotation_mpii = matlab_mpii.__dict__['annolist'][0].shape[0]

Ditugun irudi kopurua ikusi eta nola banatzen diren ere:

In [10]:
print(num_images)
train = 0
pertsona_bakarra = 0
pertsona_bakar_train = 0
pertsona_bakar_test = 0
pertsona_ez = 0
for i in range(num_images):
  train += matlab_mpii.__dict__['img_train'][0, i].flatten()[0]
  if(len(matlab_mpii.__dict__['single_person'][i][0].flatten())==1):
    pertsona_bakarra += 1
    if(matlab_mpii.__dict__['img_train'][0, i].flatten()[0]==1):
      pertsona_bakar_train += 1
    else:
      pertsona_bakar_test += 1
  if(len(matlab_mpii.__dict__['single_person'][i][0].flatten())==0):
    pertsona_ez +=1

print(train)
print(pertsona_bakarra)
print(pertsona_bakar_test)
print("Datu base osoaren train%:", train/num_images)
print("Datu basearen buru 1%:", pertsona_bakarra/num_images)
print("Datu base buru 1 train%:", pertsona_bakar_train/pertsona_bakarra)
print("Pertsonarik gabe argazkien %:", pertsona_ez/num_images)

24987
18079
16814
4569
Datu base osoaren train%: 0.7235362388441989
Datu basearen buru 1%: 0.6729099131548405
Datu base buru 1 train%: 0.7282621624836446
Pertsonarik gabe argazkien %: 0.16132388842197942


Zati honetan id_x_y listan buru bat bakarrik anotatuta duten argazkien informazioa gordeko dugu. 

In [12]:
perts_bat_anotatua_train = 0
perts_bat_anotatua_test = 0
perts_bat_anotatu = 0
perts_bat_anotatu_burua_du = 0
id_x_y = []
for i in range(0,num_images):
  annotated_person_flag = False
  if(len(matlab_mpii.__dict__['single_person'][i][0].flatten())==1):
    perts_ind = matlab_mpii.__dict__['single_person'][i][0].flatten()[0]
    annotation_mpii = matlab_mpii.__dict__['annolist'][0, i]
    person = perts_ind - 1

    try:
      annopoints_img_mpii = annotation_mpii.__dict__['annorect'][0, person].__dict__['annopoints'][0, 0]
      #img = image_info[dataset_name_]['img'][i]
      img_name = annotation_mpii.__dict__['image'][0, 0].__dict__['name'][0]
      #img_gt = image_info[dataset_name_]['img_gt'][i]
      num_joints = annopoints_img_mpii.__dict__['point'][0].shape[0]
      annotated_person_flag = True

    except KeyError:
                # Person 'x' could not have annotated joints, hence move to person 'y'
        continue
      
    if(annotated_person_flag):
      
      if(matlab_mpii.__dict__['img_train'][0, i].flatten()[0]==1):
        perts_bat_anotatua_train +=1

      perts_bat_anotatu +=1  
      for j in range(num_joints):
        id_ = annopoints_img_mpii.__dict__['point'][0, j].__dict__['id'][0][0]
        if(id_==9):
          perts_bat_anotatu_burua_du += 1
          x = annopoints_img_mpii.__dict__['point'][0, j].__dict__['x'].flatten()[0]
          y = annopoints_img_mpii.__dict__['point'][0, j].__dict__['y'].flatten()[0]
          sartzeko = [img_name,x,y]
          id_x_y.append(sartzeko)
      
    
print("Anotatu_pertsona 1:",perts_bat_anotatua_train+perts_bat_anotatua_test)
print(perts_bat_anotatua_train)
print(perts_bat_anotatua_test)
print(perts_bat_anotatu)
print("Train/test anotatu_pertsona bat:",(perts_bat_anotatua_train/(perts_bat_anotatua_train+perts_bat_anotatua_test)))
print(perts_bat_anotatu_burua_du)



Anotatu_pertsona 1: 12245
12245
0
12245
Train/test anotatu_pertsona bat: 1.0
12196


In [ ]:
!mkdir '/content/bat_buru' 
k = 0
for i in id_x_y:
  nombre = '/content/pose_data/images/' + i[0]
  shutil.move(nombre, "/content/bat_buru")
  k += 1

print("Mugitutako argazki kopurua:", k)

Datuen banaketak egiteko

In [19]:
indices = np.arange(len(id_x_y))
size1 = int(len(id_x_y)*0.8) #Entrenamenduko datuen tamaina
size2 = int(size1*0.8) #Dev datuen tamaina

to_train_indices, test_indices = train_test_split(indices, train_size=size1, random_state=42)
train_indices, dev_indices = train_test_split(to_train_indices, train_size=size2, random_state=42)

train_set = Subset(id_x_y, train_indices)
dev_set = Subset(id_x_y, dev_indices)
test_set = Subset(id_x_y, test_indices)


print("Train datu kopurua:", len(train_set), "Dev datu kopurua:", len(dev_set), "Test datu kopurua:", len(test_set))
denak_tam = len(train_set) + len(dev_set) + len(test_set)
print(denak_tam)

Train datu kopurua: 7804 Dev datu kopurua: 1952 Test datu kopurua: 2440
12196


Banatu egingo ditugu Train/Dev/Test karpeta desberdinetan:

In [20]:
!mkdir '/content/train'
!mkdir '/content/test'
!mkdir '/content/dev'

for i in train_set:
  nombre = '/content/bat_buru/' + i[0]
  shutil.move(nombre, "/content/train")

for i in test_set:
  nombre = '/content/bat_buru/' + i[0]
  shutil.move(nombre, "/content/test")

for i in dev_set:
  nombre = '/content/bat_buru/' + i[0]
  shutil.move(nombre, "/content/dev")

Argazkiak gorde:
#Direktorio propioak jarri

In [23]:
# Direktorio propioak jarri
!zip -r /content/drive/MyDrive/AA3/IANS/P73_Pose_Estimation_DL_D3.pdf/Entregatzeko/dev.zip /content/dev
!zip -r /content/drive/MyDrive/AA3/IANS/P73_Pose_Estimation_DL_D3.pdf/Entregatzeko/test.zip /content/test
!zip -r /content/drive/MyDrive/AA3/IANS/P73_Pose_Estimation_DL_D3.pdf/Entregatzeko/train.zip /content/train

Se han truncado las últimas 5000 líneas del flujo de salida.
  adding: content/train/017087043.jpg (deflated 0%)
  adding: content/train/057450433.jpg (deflated 1%)
  adding: content/train/068723841.jpg (deflated 0%)
  adding: content/train/024029912.jpg (deflated 5%)
  adding: content/train/011330350.jpg (deflated 0%)
  adding: content/train/053157215.jpg (deflated 1%)
  adding: content/train/046267561.jpg (deflated 0%)
  adding: content/train/005166872.jpg (deflated 0%)
  adding: content/train/000568822.jpg (deflated 0%)
  adding: content/train/076231876.jpg (deflated 2%)
  adding: content/train/064265083.jpg (deflated 1%)
  adding: content/train/034488917.jpg (deflated 0%)
  adding: content/train/013497849.jpg (deflated 0%)
  adding: content/train/099769622.jpg (deflated 0%)
  adding: content/train/006272159.jpg (deflated 1%)
  adding: content/train/041081640.jpg (deflated 1%)
  adding: content/train/089605669.jpg (deflated 0%)
  adding: content/train/040919664.jpg (deflated 0%)
  a

Aragazkien informazioa gordetzeko

In [30]:
with open('/content/drive/MyDrive/AA3/IANS/P73_Pose_Estimation_DL_D3.pdf/Entregatzeko/dev_set.txt', 'w') as f:
    for line in dev_set:
        f.write(f"{line}\n")

with open('/content/drive/MyDrive/AA3/IANS/P73_Pose_Estimation_DL_D3.pdf/Entregatzeko/train_set.txt', 'w') as f:
    for line in train_set:
        f.write(f"{line}\n")       

with open('/content/drive/MyDrive/AA3/IANS/P73_Pose_Estimation_DL_D3.pdf/Entregatzeko/test_set.txt', 'w') as f:
    for line in test_set:
        f.write(f"{line}\n")

In [31]:
import csv 
    
# field names 
fields = ['Argazkia', 'x', 'y'] 
    
# data rows of csv file 
rows = []
with open('/content/drive/MyDrive/AA3/IANS/P73_Pose_Estimation_DL_D3.pdf/Entregatzeko/test_set.txt') as f:
    for line in f.read().splitlines():
      banatua = line.split(",")
      lista = [banatua[0][2:-1], float(banatua[1][1:]), float(banatua[2][1:-1])]
      rows.append(lista)
    
# Gordetzeko csv-aren direktorioa test.csv
filename = "/content/drive/MyDrive/AA3/IANS/P73_Pose_Estimation_DL_D3.pdf/Entregatzeko/test.csv"
    
# writing to csv file 
with open(filename, 'w') as csvfile: 
    # creating a csv writer object 
    csvwriter = csv.writer(csvfile) 
        
    # writing the fields 
    csvwriter.writerow(fields) 
        
    # writing the data rows 
    csvwriter.writerows(rows)

In [32]:
import csv 
    
# field names 
fields = ['Argazkia', 'x', 'y'] 
    
# data rows of csv file 
rows = []
with open('/content/drive/MyDrive/AA3/IANS/P73_Pose_Estimation_DL_D3.pdf/Entregatzeko/train_set.txt') as f:
    for line in f.read().splitlines():
      banatua = line.split(",")
      lista = [banatua[0][2:-1], float(banatua[1][1:]), float(banatua[2][1:-1])]
      rows.append(lista)
    
# Gordetzeko csv-aren direktorioa train.csv 
filename = "/content/drive/MyDrive/AA3/IANS/P73_Pose_Estimation_DL_D3.pdf/Entregatzeko/train.csv"
    
# writing to csv file 
with open(filename, 'w') as csvfile: 
    # creating a csv writer object 
    csvwriter = csv.writer(csvfile) 
        
    # writing the fields 
    csvwriter.writerow(fields) 
        
    # writing the data rows 
    csvwriter.writerows(rows)

In [33]:
import csv 
    
# field names 
fields = ['Argazkia', 'x', 'y'] 
    
# data rows of csv file 
rows = []
with open('/content/drive/MyDrive/AA3/IANS/P73_Pose_Estimation_DL_D3.pdf/Entregatzeko/dev_set.txt') as f:
    for line in f.read().splitlines():
      banatua = line.split(",")
      lista = [banatua[0][2:-1], float(banatua[1][1:]), float(banatua[2][1:-1])]
      rows.append(lista)
    
# Gordetzeko csv-aren direktorioa dev.csv
filename = "/content/drive/MyDrive/AA3/IANS/P73_Pose_Estimation_DL_D3.pdf/Entregatzeko/dev.csv"
    
# writing to csv file 
with open(filename, 'w') as csvfile: 
    # creating a csv writer object 
    csvwriter = csv.writer(csvfile) 
        
    # writing the fields 
    csvwriter.writerow(fields) 
        
    # writing the data rows 
    csvwriter.writerows(rows)